# Elasticsearch

![Elasticsearch](images/Elasticsearch_logo.png)

> Elasticsearch is a distributed, free and open search and analytics engine for all types of data, including textual, numerical, geospatial, structured, and unstructured. Elasticsearch is built on Apache Lucene [...]

>  Elasticsearch is the central component of the Elastic Stack, a set of free and open tools for data ingestion, enrichment, storage, analysis, and visualization. Commonly referred to as the ELK Stack (after Elasticsearch, Logstash, and Kibana), the Elastic Stack now includes a rich collection of lightweight shipping agents known as Beats for sending data to Elasticsearch.

![Apache Lucene](images/Apache_Lucene_logo.png)

> Lucene Core is a Java library providing powerful indexing and search features, as well as spellchecking, hit highlighting and advanced analysis/tokenization capabilities.

- ranked searching -- best results returned first
- many powerful query types: phrase queries, wildcard queries, proximity queries, range queries and more
- fielded searching (e.g. title, author, contents)
- nearest-neighbor search for high-dimensionality vectors
- sorting by any field
- multiple-index searching with merged results
- allows simultaneous update and searching
- flexible faceting, highlighting, joins and result grouping
- fast, memory-efficient and typo-tolerant suggesters
- pluggable ranking models, including the Vector Space Model and Okapi BM25
- configurable storage engine (codecs)

## Index

De grootste eenheid van gegevens in Elasticsearch

Indices zijn logische partities van documenten en kunnen worden vergeleken met een database in de wereld van relationele databases.

## Shard

Shards zijn een enkele Lucene index.

Shards zijn de bouwstenen van Elasticsearch en maken schaalbaarheid mogelijk.

## Replica

Replicas zijn kopieën van shards.

Replicas accepteren ook leesverzoeken, dus het toevoegen van replicas kan helpen om de zoekprestaties te verbeteren.

Om hoge beschikbaarheid te garanderen, worden replica's niet op dezelfde node geplaatst als de oorspronkelijke shards (de "primary" shard genoemd) van waaruit ze zijn gerepliceerd.

## Shard allocatie

- primary
- replica

[Elastic Calculator](https://gbaptista.github.io/elastic-calculator/)

## Sharding strategie

Afhankelijk van de use case :(

- Zoekopdrachten worden uitgevoerd op een enkele thread per shard (te veel shards en de threadpool raakt uitgeput)
- Elke index en shard heeft overhead (en gebruikt JVM heap geheugen)
- Elasticsearch balanceert automatisch shards binnen een *data tier* (hoeveel redundantie is nodig)

## Sharding best practices

Een paar overwegingen:

- Verwijder indices, geen documenten (verwijderde documenten worden gemarkeerd als verwijderd en blijven geheugen gebruiken)
- Streef naar shardgroottes tussen 10GB en 50GB (grotere shards hebben meer tijd nodig om te herstellen na een storing)
- Streef naar 20 shards of minder per GB heapgeheugen (het aantal shards dat een node kan bevatten is evenredig met het heapgeheugen van het node)
- Vermijd *hotspots* op nodes (onevenwichte verdeling van shards over nodes)

## Nodes

Nodes weten van alle andere nodes in het cluster en kunnen verzoeken van cliënten naar de juiste node doorsturen.

Nodes hebben één of meerdere *rollen*, onder andere

- `master`
- `data`
- `data_content`
- `data_hot`
- `ingest`
- `ml`

Elk cluster heeft de volgende node rollen nodig:

- `master`
- `data_content` en `data_hot`
- OF `data`

### `master`

*Master-eligible node*

Een node die de masterrol heeft, waardoor het in aanmerking komt om te worden gekozen als het masternode, dat het cluster bestuurt.

### `data`

*Data node*

Een node die de data rol heeft. Nodes bevatten gegevens en voeren gegevensgerelateerde operaties uit, zoals CRUD, zoeken en aggregaties. Een node met de data rol kan een van de gespecialiseerde data node rollen vervullen.

### `data_node`

*Content data node*

Content tier nodes zijn gewoonlijk geoptimaliseerd voor query performance.

Over het algemeen voor verzamelingen van items, zoals een productcatalogus of een artikelarchief.

- waarde van de inhoud blijft in de loop van de tijd relatief constant
- hebben typisch lange data retentie vereisten
- items snel kunnen ophalen ongeacht hoe oud ze zijn

Data nodes geven voorrang aan verwerkingskracht boven IO throughput, zodat ze complexe zoekopdrachten en aggregaties kunnen verwerken en snel resultaten kunnen teruggeven.

### `data_hot`

*Hot data node*

Hot tier nodes zijn gewoonlijk geoptimaliseerd voor snel lezen en schrijven

Over het algemeen voor timeseries data.

- bevat meest recente en meest frequent gezochte tijdreeks data

Vereist meer hardware resources en snellere opslag (SSDs).

### Scatter / gather

Een zoekopdracht wordt uitgevoerd in twee fasen die worden gecoördineerd door de node die het verzoek ontvangt - de *coordinating node*

- in *scatter* fase wordt het verzoek doorgestuurd naar andere nodes
- in *gather* fase worden resultaten verzameld

Elke node is impliciet een coördinerende node.

Een node *expliciet* zonder rollen zal alleen optreden als een coördinerende node. Een dergelijke node zal voldoende geheugen en CPU moeten hebben om de *gather* fase aan te kunnen.

## Cluster

Een logische verzameling van nodes (that's it!).